In [ ]:
import ast
import defcon
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from data_utils import parse_list, glyph_to_img,\
    glyph_stats, glyph_to_svg_path, normalize_glyph,\
    transform_svg, svg_format

df = pd.read_csv("ufo_data.csv", converters={"subsets": parse_list, "variants":ast.literal_eval})

In [ ]:
# Initialize an empty list to store the glyph DataFrames
glyph_dfs = []

import os
if not os.path.exists("glyphs"):
    os.mkdir("glyphs")
    
# Iterate through each font family and variant in the font_dataset DataFrame
#for _, row in tqdm(df[:10].iterrows(), total=df.shape[0], desc="Extracting Glyphs..."):
for _, row in tqdm(df[:2].iterrows(), total=2):
    family = row['family']
    variants = row['variants']

    for variant, ufo_file_path in variants.items():
        # Open the UFO file for the font
        font = defcon.Font(ufo_file_path)

        info = {
            'ascender' : font.info.ascender,
            'capHeight' : font.info.capHeight,
            'descender' : font.info.descender,
            'italicAngle' : font.info.italicAngle,
            'xHeight' : font.info.xHeight,
            'unitsPerEm' : font.info.unitsPerEm,
        }

        # Initialize an empty list to store the glyph data for the font
        glyph_data = []

        # Iterate through each glyph in the font
        for glyph_name in font.keys():
            # Get the glyph object for the glyph
            glyph = font[glyph_name]

            svg = glyph_to_svg_path(glyph)

            meanX, meanY, stddevX, stddevY = glyph_stats(glyph)

            #glyph_to_img(glyph, "glyph.png")

            
            # Create a dictionary to store the glyph data
            glyph_dict = {
                'family' : family,
                'variant' : variant,
                'glyph_name': glyph_name,
                'svg': svg,
                'advance': glyph.width,
                'unicode' : glyph.unicode,
                'meanX' : meanX,
                'meanY' : meanY,
                'stddevX' : stddevX,
                'stddevY' : stddevY,
                'area' : glyph.area,
                'bounds' : glyph.bounds
            }
            # Add the glyph data to the list
            glyph_data.append(glyph_dict)

        # Create a glyph DataFrame for the font
        glyph_df = pd.DataFrame(glyph_data)
        # Add the glyph DataFrame to the list of glyph DataFrames
        glyph_dfs.append(glyph_df)

# Concatenate the glyph DataFrames into a single DataFrame
glyphs_df = pd.concat(glyph_dfs)


In [ ]:
glyphs_df = pd.concat(glyph_dfs)


In [ ]:
# Normalize contours
glyphs_df = glyphs_df.drop(glyphs_df[glyphs_df.stddevY == 0].index)
tqdm.pandas(desc="Normalizing Svgs...", postfix=None)
glyphs_df["n_svg"] = glyphs_df.progress_apply(normalize_glyph, axis=1)

In [ ]:
# Sample Polygons


In [ ]:
# Raster Images

In [ ]:
glyphs_df.to_csv("glyphs_data.csv", index=False)

In [ ]:
from IPython.core.display import HTML
s = glyphs_df.svg.iloc[100]
s = transform_svg(s, 0, -1400, .1, -.1)
display(HTML(svg_format % s))